# OpenVisus Commons Notebook

<table width='100%'>
<tr style="background-color:#FFFFFF;">
<td><img src="https://nationalsciencedatafabric.org/assets/images/logo.png" style="width:260px;"/></td>
<td><img src="http://cedmav.org/templates/rt_requiem/custom/images/logo-gr.png" style="width:200px;"/></td>
<td><img src="http://prisms-center.org/assets/img/sc-prisms-logo.png" style="width:200px;"/></td>
<td>
Created by:<br>
 <a href='https://nationalsciencedatafabric.org/'/>National Science Data Fabric</a><br>
<a href='http://cedmav.com'>CEDMAV</a><br>
<a href='http://prisms-center.org/'>PRISMS Center</a><br>
</td>
</tr>
</table>


This notebook shows access to the OpenVisus dataset in [Jupyter notebook](https://jupyter.org/) and [Bokeh](https://bokeh.org/).

This code can be run within a self-contained Jupyter Notebook or as a webpage.
- For the first case just continue to execute the following cells to the end.
- For the second case you need to
- For the second case you need to
    - convert the notebook to a regular python file (`jupyter nbconvert --to script <filename>.ipynb`) and
    - start the Bokeh server to show the interactive webpage (` python -m bokeh serve  --show <filename.py>`)


# Python Setup

Install a recent version of python (>=3.6). For example you can use [miniforge](https://github.com/conda-forge/miniforge):

```
curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh"
bash Mambaforge-$(uname)-$(uname -m).sh
```

After the installation, remember to logout/login so that conda can properly set your  variables in your shell environment.

Create a virtual env and install packages (change the python version as needed):

```
conda create -n myenv  python=3.10
```

Activate the environment:

```
conda activate myenv
```

Check the python path, it should be inside your `${HOME}`:

```
python --version
```

Install a minimal set of packages, this list can be changed as needed:

```
python -m pip install --upgrade pip
python -m pip install jupyter jupyterlab bokeh jupyter-bokeh pandas panel numpy urllib3 matplotlib ipywidgets boto3 awscli pillow colorcet boto3 OpenVisusNoGui
```

# Notebook Setup

Import python packages:

In [ ]:
import os, sys, logging

import OpenVisus

# this may be dangerous, only for local testing/debugging
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*"

# if you want more debug info
# os.environ["VISUS_CPP_VERBOSE"]="1"
# os.environ["VISUS_AWS_VERBOSE"]="1"
# os.environ["BOKEH_LOG_LEVEL"]="debug" 

import bokeh
import bokeh.io
import bokeh.models.widgets
import bokeh.core.validation
import bokeh.plotting
import bokeh.core.validation.warnings
import bokeh.layouts

bokeh.core.validation.silence(bokeh.core.validation.warnings.EMPTY_LAYOUT, True)
bokeh.core.validation.silence(bokeh.core.validation.warnings.FIXED_SIZING_MODE, True)

bokeh.io.output_notebook()

# Test if Bokeh is working

In [ ]:
def bokeh_test_app(doc):
    fig = bokeh.plotting.figure(title="Multiple line example", x_axis_label="x", y_axis_label="y", height=200)
    fig.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5])
    grid = bokeh.layouts.grid(children=[fig], nrows=1, ncols=1, sizing_mode='stretch_width')
    main_layout = bokeh.layouts.column(children=[], sizing_mode='stretch_width')
    main_layout.children.append(grid)
    button = bokeh.models.widgets.Button(label="Bokeh is working", sizing_mode='stretch_width')
    button.on_click(lambda: print("Bokeh on_click event is working"))
    main_layout.children.append(button)
    doc.add_root(main_layout)


bokeh.io.show(bokeh_test_app)

# OpenVisus commons and App Setup

Import OpenVisus and load dataset

In [3]:
import OpenVisus as ov
from OpenVisus.dashboards import Slice, Slices

os.environ["VISUS_NETSERVICE_VERBOSE"] = "0"
os.environ["AWS_ACCESS_KEY_ID"] = "any"
os.environ["AWS_SECRET_ACCESS_KEY"] = "any"
ds_url = "{ds-url-here}"

# Example of accessing data with an S3 profile


In [ ]:
import os, sys

#os.environ["VISUS_CPP_VERBOSE"]="1"
#os.environ["VISUS_AWS_VERBOSE"]="1"

os.environ["AWS_ACCESS_KEY_ID"] = "any"
os.environ["AWS_SECRET_ACCESS_KEY"] = "any"

# do the whole read only for small dataset
db = OpenVisus.LoadDataset(ds_url)
data = db.read()

# show the image
from matplotlib import pyplot as plt

plt.imshow(data, interpolation='nearest')
plt.show()

# View Dataset as Single Slice

In [ ]:
def ViewSingleSlice3DApp(doc):
    db=ov.LoadDataset(ds_url)
    slice=Slice(doc,sizing_mode='stretch_width')
    slice.show_options=["palette","timestep","field","direction","offset"]
    slice.setDataset(db, direction=2)
    slice.setPalette("Greys256", palette_range=(0,255))
    slice.setTimestep(db.getTime())
    slice.setField(db.getField().name)
    doc.add_root(slice.layout)

bokeh.io.show(ViewSingleSlice3DApp)

# View Dataset as Multiple Slices

In [ ]:
def ViewMultipleSlices2DApp(doc):
    db=ov.LoadDataset(ds_url)
    slices=Slices(doc,sizing_mode='stretch_width')
    slices.setPalette("Greys256", palette_range=(0,255))
    slices.setTimestep(db.getTime())
    slices.setField(db.getField().name)
    doc.add_root(slices.layout)
    doc.add_timeout_callback(lambda: slices.setDataset(db,layout=2),100)  # IMPORTANT to fix layout

bokeh.io.show(ViewMultipleSlices2DApp)

# Visualizer and Explorer
Visualize and explore your image.

In [ ]:
import panel as pn

pn.extension('vtk')

dims = 700

db = OpenVisus.LoadDataset(ds_url)

# Change max_resolution depending on how much resolution you want. The value must be
# between 0 and db.max_resolution.
max_resolution = db.max_resolution

values = db.read(max_resolution=max_resolution, time=0)

volume = pn.panel(values, sizing_mode='stretch_both', height=200, orientation_widget=True,
                  display_slices=True, spacing=(1, 1, 1), controller_expanded=False)
slider = pn.widgets.IntSlider(name='Experiment', start=0, end=1, step=1, value=0)
slideri = pn.widgets.IntSlider(name='i-slice', start=0, end=values.shape[0], value=values.shape[0] // 2)
sliderj = pn.widgets.IntSlider(name='j-slice', start=0, end=values.shape[1], value=values.shape[1] // 2)
sliderk = pn.widgets.IntSlider(name='k-slick', start=0, end=values.shape[2], value=values.shape[2] // 2)


@pn.depends(slider)
def cb_slider(value):
    global volume
    volume.object = db.read(max_resolution=max_resolution, time=value)
    return " "


@pn.depends(slideri)
def cb_slideri(value):
    global volume
    volume.slice_i = value
    return " "


@pn.depends(sliderj)
def cb_sliderj(value):
    global volume
    volume.slice_j = value
    return " "


@pn.depends(sliderk)
def cb_sliderk(value):
    global volume
    volume.slice_k = value
    return " "


checkbox_volume = pn.widgets.Checkbox(name='Show Volume', value=True)


@pn.depends(checkbox_volume)
def cb_checkbox_volume(value):
    global volume
    volume.display_volume = value
    return " "


checkbox_slices = pn.widgets.Checkbox(name='Show Slices', value=True)


@pn.depends(checkbox_slices)
def cb_checkbox_slices(value):
    global volume
    volume.display_slices = value
    return " "


pn.Column(slider, slideri, sliderj, sliderk,
          pn.Row(checkbox_volume, checkbox_slices),
          volume, cb_slider, cb_slideri, cb_sliderj, cb_sliderk, cb_checkbox_volume, cb_checkbox_slices,
          width=dims, height=1000,
          css_classes=['panel-widget-box'],
          sizing_mode='stretch_width',
          width_policy='max')

# Run the OpenVisus Viewer

You can run the OpenVisus viewer on your system. The viewer has many more features than are available in this notebook.
To run the viewer copy execute the block below. It will show you the command to run for this dataset.

In [ ]:
import sys
print(f'{sys.executable} -m OpenVisus viewer --dataset {ds_url}')

---------------------------
Copyright (c) 2023 **National Science Data Fabric (NSDF)** 

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

**The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.** 

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.